In [3]:
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

load_dotenv()

password = os.getenv('PASSWORD')
username = os.getenv('USER')
host = os.getenv('HOST')
port = os.getenv('PORT')

def rentals_month(engine, month, year):
    query = f"""
    SELECT customer_id, COUNT(*) AS rentals_{month:02d}_{year}
    FROM rental
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year}
    GROUP BY customer_id
    """
    df = pd.read_sql(query, engine)
    return df

def rental_count_month(df, month, year):
    df.rename(columns={df.columns[1]: f"rentals_{month:02d}_{year}"}, inplace=True)
    return df

def compare_rentals(df1, df2):
    combined_df = pd.merge(df1, df2, on='customer_id', how='inner')
    column_names = combined_df.columns[1:3]
    combined_df['difference'] = combined_df[column_names[0]] - combined_df[column_names[1]]
    return combined_df

engine = create_engine('mysql+mysqlconnector://' + username + ':' + password + '@' + host + ':' + port + '/sakila')


rentals_may = rentals_month(engine, 5, 2005)
rentals_june = rentals_month(engine, 6, 2005)

rentals_may_count = rental_count_month(rentals_may, 5, 2005)
rentals_june_count = rental_count_month(rentals_june, 6, 2005)

comparison = compare_rentals(rentals_may_count, rentals_june_count)
display(comparison.head())

,customer_id,rentals_05_2005,rentals_06_2005,difference
0,1,2,7,-5
1,2,1,1,0
2,3,2,4,-2
3,5,3,5,-2
4,6,3,4,-1
